Importing Libraries

In [125]:
import zipfile
import pandas as pd
import ast 
import pickle  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Extracting and Loading Datasets

In [127]:
credits_zip_path="tmdb_5000_credits.csv.zip"
movies_zip_path="tmdb_5000_movies.csv.zip"


In [128]:
extract_dir="data_csv"

In [129]:
with zipfile.ZipFile(credits_zip_path, 'r') as f:
    f.extractall(extract_dir)
with zipfile.ZipFile(movies_zip_path, 'r') as f:
    f.extractall(extract_dir)

In [130]:
df_credits=pd.read_csv("data_csv/tmdb_5000_credits.csv")
df_movies=pd.read_csv("data_csv/tmdb_5000_movies.csv")


Data Preprocessing

In [132]:
df_credits.sample(5)

,movie_id,title,cast,crew
4778,38780,Rampage,"[{""cast_id"": 1, ""character"": ""Bill Williamson""...","[{""credit_id"": ""52fe46df9251416c91062979"", ""de..."
4034,76706,Christmas Mail,"[{""cast_id"": 1, ""character"": ""Kristi North"", ""...","[{""credit_id"": ""52fe4949c3a368484e123dc7"", ""de..."
683,12138,3000 Miles to Graceland,"[{""cast_id"": 15, ""character"": ""Michael Zane"", ...","[{""credit_id"": ""52fe44bc9251416c7503f2db"", ""de..."
2287,26367,I Can Do Bad All By Myself,"[{""cast_id"": 1, ""character"": ""Madea"", ""credit_...","[{""credit_id"": ""52fe44fdc3a368484e04271b"", ""de..."
3444,621,Grease,"[{""cast_id"": 17, ""character"": ""Danny Zuko"", ""c...","[{""credit_id"": ""53807b97c3a36805a1004d86"", ""de..."


In [133]:
df_movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2700,14000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,2013,"[{""id"": 1731, ""name"": ""writing""}, {""id"": 1808,...",fr,Le scaphandre et le papillon,"Elle France editor Jean-Dominique Bauby, who, ...",13.654940,"[{""name"": ""France 3 Cin\u00e9ma"", ""id"": 591}, ...","[{""iso_3166_1"": ""FR"", ""name"": ""France""}, {""iso...",2007-05-22,19777647,112.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}]",Released,Let your imagination set you free,The Diving Bell and the Butterfly,7.4,283
3794,4000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,54580,"[{""id"": 514, ""name"": ""spain""}, {""id"": 1010, ""n...",es,Los lunes al sol,After the closure of their shipyard in Norther...,3.663108,"[{""name"": ""V\u00eda Digital"", ""id"": 357}, {""na...","[{""iso_3166_1"": ""ES"", ""name"": ""Spain""}, {""iso_...",2002-09-27,146402,113.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,This film is not based on a real story. It is ...,Mondays in the Sun,7.1,36
3392,3500000,"[{""id"": 9648, ""name"": ""Mystery""}, {""id"": 18, ""...",NaN,1956,"[{""id"": 1333, ""name"": ""desperation""}, {""id"": 3...",en,Gerry,Two friends named Gerry become lost in the des...,5.038953,"[{""name"": ""Epsilon Motion Pictures"", ""id"": 117...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-01-12,0,103.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Gerry,6.2,68
2012,23000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,11632,"[{""id"": 212, ""name"": ""london england""}, {""id"":...",en,Vanity Fair,"Beautiful, funny, passionate, and calculating,...",6.618149,"[{""name"": ""Alliance Films"", ""id"": 2514}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2004-09-01,16123851,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,"On September 1st, a heroine will rise.",Vanity Fair,5.5,73
122,150000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",http://www.x-menorigins.com/,2080,"[{""id"": 417, ""name"": ""corruption""}, {""id"": 185...",en,X-Men Origins: Wolverine,"After seeking to live a normal life, Logan set...",5.954334,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-04-28,373062864,107.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Witness the Origin.,X-Men Origins: Wolverine,6.2,4021


In [134]:
df_movies.shape

(4803, 20)

In [135]:
df_credits.shape

(4803, 4)

In [136]:
movies = df_movies.merge(df_credits,on='title')

In [137]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [138]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [139]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [140]:
import ast

In [141]:
def convert(text):     # Function for changing dictionary of genres in the form of text 
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [142]:
movies.dropna(inplace=True)

In [143]:
movies['genres'] = movies['genres'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [144]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [145]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [146]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [147]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [148]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [149]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [150]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [151]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2142,20391,Paint Your Wagon,A Michigan farmer and a prospector form a part...,"[Drama, Action, Comedy, Western, Music]","[mining, prospector]","[Lee Marvin, Clint Eastwood, Jean Seberg]",[Joshua Logan]
4699,289180,H.,H. is a modern interpretation of a classic Gre...,"[Thriller, Drama, Science Fiction]",[woman director],"[Robin Bartlett, Rebecca Dayan, Will Janowitz]","[Rania Attieh, Daniel Garcia]"
3905,20737,Air Bud,A young boy and a talented stray dog with an a...,"[Comedy, Family, Drama]","[clown, golden retriever, dog actor]","[Kevin Zegers, Wendy Makkena, Michael Jeter]",[Charles Martin Smith]
1209,209451,Jersey Boys,From director Clint Eastwood comes the big-scr...,"[Music, Drama]","[biography, based on play]","[Christopher Walken, Vincent Piazza, Freya Tin...",[Clint Eastwood]
376,10357,Volcano,An earthquake shatters a peaceful Los Angeles ...,"[Science Fiction, Action, Drama, Thriller]","[subway, lava, volcano, volcanologist, los ang...","[Tommy Lee Jones, Anne Heche, Gaby Hoffmann]",[Mick Jackson]


In [152]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [153]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [154]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [155]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [156]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [157]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [158]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


Model Building

In [160]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

In [161]:
vector = cv.fit_transform(new['tags']).toarray()

In [162]:
vector.shape

(4806, 5000)

In [163]:
from sklearn.metrics.pairwise import cosine_similarity

In [164]:
similarity = cosine_similarity(vector)

In [165]:
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [166]:
new[new['title'] == 'The Lego Movie'].index[0]

744

Model Prediction

In [168]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
        
    

In [169]:
recommend('Gandhi')

Gandhi, My Father
The Wind That Shakes the Barley
A Passage to India
Guiana 1838
Ramanujan


Model Serialization

In [171]:
import pickle

In [172]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))